In [1]:
# Here we take care of paths.
# Make sure root project directory is named 'VESUVIUS_Challenge' for this to work

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in the root folder of the project
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [2]:
import torch
import monai
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import Tuple, List
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Dataset import Vesuvius_Tile_Datamodule
from lit_models.Vesuvius_Lit_Model import Lit_Model
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn as nn

from Models.PreBackbone_3d_SIMPLE import PreBackbone_3D_SIMPLE
import torch.nn as nn
from functools import partial
import torchvision
import torch.nn.functional as F


from lit_models.scratch_models import FPNDecoder
from segmentation_models_pytorch.base import SegmentationHead


2023-05-22 14:31:09,508 - Created a temporary directory at /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpeap5oz1j
2023-05-22 14:31:09,508 - Writing /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpeap5oz1j/_remote_module_non_scriptable.py


In [3]:
PATCH_SIZE = 256
Z_DIM = 16
COMPETITION_DATA_DIR_str =  "kaggle/input/vesuvius-challenge-ink-detection/"


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

# change to the line below if not using Apple's M1 or chips
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
class Model_3dAtt_w_Segformer(nn.Module):
    def __init__(self ):
        
        super().__init__()
        
        self.model_3d = PreBackbone_3D_SIMPLE(z_dim=Z_DIM).to(DEVICE) 
        
       
        self.head = nn.AdaptiveAvgPool3d((1, None, None))

    def forward(self, x):
      
        outs_3d = self.model_3d(x)
        logits = self.model_2d(outs_3d)
        
       
       
            
        
        return logits













In [5]:


class CFG:
    
    device = DEVICE
    
    THRESHOLD = 0.4
    use_wandb = True
    
    ######### Dataset #########
    
    # stage: 'train' or 'test'
    stage = 'train' 
    
    # location of competition Data
    competition_data_dir = COMPETITION_DATA_DIR_str
    
    # Number of slices in z-dim: 1<z_dim<65
    z_dim = Z_DIM
    
    # fragments to use for training avalaible [1,2,3]
    train_fragment_id=[2,3]
    
    # fragments to use for validation
    val_fragment_id=[1]
    
    

    
    
    batch_size = 8
    
    # Size of the patch and stride for feeding the model
    patch_size = PATCH_SIZE
    stride = patch_size // 2
    
    
    num_workers = 8
    on_gpu = True
    
    
    ######## Model and Lightning Model paramters ############
    
    # MODEL
    model = Model_3dAtt_w_Segformer().to(DEVICE) 
    
    
    
    
    checkpoint = None
    save_directory = None
    
    
    accumulate_grad_batches = 128 // batch_size  # experiments showed batch_size * accumulate_grad = 192 is optimal
    learning_rate =  0.00002
    eta_min = 1e-8
    t_max = 50
    max_epochs = 120
    weight_decay =  0.001
    precision =16
    
    # checkpointing
    save_top_k=5
    
    monitor="FBETA"
    mode="max"
    
    
    ####### Augemtnations ###############
    
    # Training Aug
    train_transforms = [
        # A.RandomResizedCrop(
        #     size, size, scale=(0.85, 1.0)),
        A.Resize(patch_size, patch_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.75),
        A.ShiftScaleRotate(p=0.75),
        A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(),
                A.MotionBlur(),
                ], p=0.4),
        
       
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        A.CoarseDropout(max_holes=1, max_width=int(patch_size * 0.3), max_height=int(patch_size * 0.3), 
                        mask_fill_value=0, p=0.5),
        # A.Cutout(max_h_size=int(size * 0.6),
        #          max_w_size=int(size * 0.6), num_holes=1, p=1.0),
        A.Normalize(
            mean= [0] * z_dim,
            std= [1] * z_dim
        ),
        ToTensorV2(transpose_mask=True),
    ]
    

    
    # Validaiton Aug
    val_transforms = [
        A.Resize(patch_size, patch_size),
        A.Normalize(
            mean= [0] * z_dim,
            std= [1] * z_dim
        ),
        ToTensorV2(transpose_mask=True),
    ]
    
    # Test Aug
    test_transforms = [
        A.Resize(patch_size, patch_size),
        A.Normalize(
            mean=[0] * z_dim,
            std=[1] * z_dim
        ),

        ToTensorV2(transpose_mask=True),
    ]
        
    
    

In [6]:
dataset = Vesuvius_Tile_Datamodule(cfg=CFG)#.to(DEVICE)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [7]:
lit_model = Lit_Model(cfg=CFG,).to(DEVICE) 

Checkpoint = False
if Checkpoint:
    lit_model = lit_model.load_from_checkpoint('logs/Model_48_3d_nonorm_w_mitb3_bce50_05tver60/last.ckpt',
                                               #learning_rate =7e-6 ,
                                                #t_max = 70,
                                               #eta_min = 1e-8,
                                               #weight_decay =  0.0001,
                                              )


wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [ ]:
SAVE_DIR = 'logs/Pre_Model3d_only_05bce3+05tver'

checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor="FBETA",
    mode="max",
    dirpath=SAVE_DIR,
    filename="Pre_Model3d_only_05bce3{epoch:02d}{FBETA:.2f}{val_loss:.2f}{fbeta_4:.2f}{recall:.2f}{precision:.2f}",
    save_last =True,
)

torch.autograd.set_detect_anomaly(True)
trainer = pl.Trainer(
        accelerator='gpu',
        #benchmark=True,
        max_epochs=CFG.max_epochs,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir=SAVE_DIR),
        log_every_n_steps=1,
        default_root_dir = SAVE_DIR,
        #overfit_batches=1,
        #precision= CFG.precision,
        accumulate_grad_batches=CFG.accumulate_grad_batches, 
        callbacks=[checkpoint_callback],
        #gradient_clip_val=1,
        #resume_from_checkpoint ='logs/Model_48_3dAttn_w_FPN_mitb3_bce75_05tver60/last.ckpt'
        #detect_anomaly=True
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/gcp_checkpoints/MoUB4_Bce015_Tver_alpha085epoch_64.ckpt'
           )

2023-05-22 14:31:34,116 - GPU available: True (mps), used: True
2023-05-22 14:31:34,117 - TPU available: False, using: 0 TPU cores
2023-05-22 14:31:34,118 - IPU available: False, using: 0 IPUs
2023-05-22 14:31:34,118 - HPU available: False, using: 0 HPUs
Adjusting learning rate of group 0 to 2.0000e-05.
2023-05-22 14:31:34,215 - 
  | Name         | Type                    | Params
---------------------------------------------------------
0 | metrics      | ModuleDict              | 0     
1 | model        | Model_3dAtt_w_Segformer | 301 K 
2 | loss_tversky | TverskyLoss             | 0     
3 | loss_bce     | SoftBCEWithLogitsLoss   | 0     
---------------------------------------------------------
301 K     Trainable params
0         Non-trainable params
301 K     Total params
1.205     Total estimated model params size (MB)


/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /Users/gregory/PROJECT_ML/VESUVIUS_Challenge/logs/Pre_Model3d_only_05bce3+05tver exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Sanity Checking: 0it [00:00, ?it/s]

import torch
import torch.nn as nn
from einops import rearrange


# TODO: implement attnetion between slices. currently only local info


# TODO: try using Depth-wise convolution https://medium.com/@zurister/depth-wise-convolution-and-depth-wise-separable-convolution-37346565d4ec
# TODO:


class PreBackbone_3D_SIMPLE(nn.Module):
    def __init__(self, z_dim= 48, emdedding_dims=4, conv_dimensions=[16,32,64], filter_sizes=[16, 32, 64], batch_norm=False):
        super(PreBackbone_3D_SIMPLE, self).__init__()

        self.embed_layer = Embed(emdedding_dims=emdedding_dims)
        self.attention = EfficientMultiHeadAttention(channels=z_dim // 2, att_dim =256)

        self.pool = nn.AvgPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1))
        self.pool_z = nn.AvgPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1))
        
        self.global_pool = nn.AdaptiveAvgPool3d((1, None, None))
        self.global_pool_final = nn.AdaptiveAvgPool3d((3, None, None))
        self.batch_norm = torch.nn.BatchNorm3d(num_features=1, momentum=0.1)
        self.leaky_relu = nn.LeakyReLU(inplace=False)

        # layer 1
        self.conv1 = nn.Conv3d(in_channels=emdedding_dims+1,
                               out_channels=conv_dimensions[0],
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               )

        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)

        # layer 2
        self.conv2 = nn.Conv3d(in_channels=conv_dimensions[0]//2,
                               out_channels=conv_dimensions[1],
                               kernel_size=3,
                               stride=1,
                               padding=1
                               )
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)

        # layer 3
        self.conv3 = nn.Conv3d(in_channels=conv_dimensions[1]//2,
                               out_channels=conv_dimensions[2],
                               kernel_size=(3,3,3),
                               stride=(1,1,1,),
                               padding=(1,1,1)
                               )

        torch.nn.init.xavier_uniform_(self.conv3.weight)
        torch.nn.init.zeros_(self.conv3.bias)

    def forward(self, x):
        x = x.unsqueeze(1)



        # Stage 1 Embedding and Z_Dim attention

        # embed layer produces tensors:
        # x_orig =  (B, emdedding_dims[0], Z/2, H, W for residual connection
        # x_att = (B, 1, Z/2, H/4, W/4)
        x_orig, x_att = self.embed_layer(x) #(B, 1, Z, H, W) -> (B, emdedding_dims[0], Z/2, H, W) and (B, 1, Z/2, H/4, W/4)
        x_orig = self.leaky_relu(x_orig)
        x_att = self.leaky_relu(x_att) 


        # attention layer for z_dim
        x_after_att = self.attention(x_att) # (B, 1, Z/2, H/4, W/4) ->  (B, 1, Z/2, H, W)

        x = torch.cat((x_orig, x_after_att), dim=1) # 

        # Stage 2 Convolutions -- 3 layers
        print(x.shape)

        # Layer 1
        y = self.conv1(x)  # (B, 1,  C, H, W) -> (B, 4,  C/2, H, W)
        print(y.shape)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.pool(y)  # (B, 4,  C/2, H, W) ->  (B, 2,  C/2, H, W)

        y = y.permute(0, 2, 1, 3, 4)
        y = self.leaky_relu(y)
     
        print('after1' ,y.shape)

        # Layer 2
        y = self.conv2(y)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.pool_z(y)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.leaky_relu(y)

        print('after2' ,y.shape)
        # Layer 3
        y = self.conv3(y)
        y = self.global_pool(y)
        y = self.leaky_relu(y)

        print('after3' ,y.shape)
        # Final pooling to 3 channels

        y = y.permute(0, 2, 1, 3, 4)
        y = self.global_pool_final(y)
        #y = y.permute(0, 2, 1, 3, 4)
        #y = self.leaky_relu(y)

        y = self.batch_norm(y)
        return y.squeeze(1)


class LayerNorm3d(nn.LayerNorm):
    def forward(self, x):
        x = rearrange(x, "b c d h w -> b d h w c")
        x = super().forward(x)
        x = rearrange(x, "b d h w c -> b c d h w")
        return x


class Embed(nn.Module):
    def __init__(self, in_channels=1, emdedding_dims=4, ):
        super().__init__()

        self.conv_3d = nn.Conv3d(in_channels=1,
                                 out_channels=emdedding_dims,
                                 kernel_size=(3, 3, 3),
                                 stride=(2, 1, 1),
                                 padding=(1, 1, 1)
                                 )
            # add a laeyr where bothz and x,y go down by half and change the next one by 2
        self.conv_3d_embed = nn.Conv3d(in_channels=emdedding_dims,
                                       out_channels=1,
                                       kernel_size=(1, 4, 4),
                                       stride=(1, 4, 4),
                                       padding=(0, 1, 1)
                                       )

        self.norm = LayerNorm3d(emdedding_dims)
        self.norm_embed = LayerNorm3d(1)

    def forward(self, x):
        x = self.conv_3d(x)
        x = self.norm(x)
        x_embed = self.conv_3d_embed(x)
        x_embed = self.norm_embed(x_embed)
        return x, x_embed


class EfficientMultiHeadAttention(nn.Module):
    def __init__(self, channels=48 // 2, att_dim =256, reduction_ratio: int = 1, num_heads: int = 8):
        super().__init__()

        self.reducer = nn.Sequential(
            nn.Conv2d(in_channels=channels,
                      out_channels=channels,
                      kernel_size=(4, 4),
                      stride=(4, 4),
                      padding=(0, 0)),

            LayerNorm_att(channels), )

        self.att = nn.MultiheadAttention(att_dim, num_heads=8, batch_first=True)

    def forward(self, x, ):
        x = x.squeeze(1)  # (b z h w)

        reduced_x = self.reducer(x)

        _, c, h, w = reduced_x.shape

        # attention needs tensor of shape (batch, sequence_length, channels)
        reduced_x = rearrange(reduced_x, "b c h w -> b  c ( h w )")
        x = rearrange(x, "b c  h w -> b  c ( h w)")
        out = self.att(reduced_x, reduced_x, reduced_x)[0]
        # reshape it back to (batch, channels, height, width)
        out = rearrange(out, "b  c (h w) -> b c h w", h=h, w=w, )
        out = nn.functional.interpolate(out, size=(256, 256), mode="bilinear", align_corners=False)
        out = out.unsqueeze(1)

        return out


class LayerNorm_att(nn.LayerNorm):
    def forward(self, x):
        x = rearrange(x, "b c h w -> b  h w c")
        x = super().forward(x)
        x = rearrange(x, "b  h w c -> b c  h w")
        return x
